In [25]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)


2.4.1


In [28]:
import numpy as np

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>


In [29]:
training_sentences = []
training_labels = []

testing_sentences=[]
testing_labels = []

for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final  = np.array(testing_labels)
print(training_labels_final)

[0 0 0 ... 0 0 1]


In [12]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token= oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [18]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a <OOV> <OOV> <OOV> when the rest of the world can go <OOV> with its own business as you <OOV> into a big <OOV> <OOV> and <OOV> for a couple of hours wonderful performances from <OOV> and <OOV> <OOV> as always <OOV> <OOV> the plot along there are no <OOV> to <OOV> no <OOV> <OOV> just a <OOV> and <OOV> <OOV> through new york life at its best a family film in every sense and one that <OOV> the <OOV> it <OOV> '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(), #or tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 16,109
Trainable params: 16,109
Non-trainable params: 0
_________________________________________________________________


In [21]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3853 - accuracy: 0.8275 - val_loss: 0.3761 - val_accuracy: 0.8290
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3851 - accuracy: 0.8270 - val_loss: 0.3795 - val_accuracy: 0.8258
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.3851 - accuracy: 0.8274 - val_loss: 0.3776 - val_accuracy: 0.8282
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.3853 - accuracy: 0.8280 - val_loss: 0.3789 - val_accuracy: 0.8248
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3845 - accuracy: 0.8282 - val_loss: 0.3826 - val_accuracy: 0.8233
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3844 - accuracy: 0.8273 - val_loss: 0.3771 - val_accuracy: 0.8272
Epoch 7/10
782/782 [==============================] - 3s 3ms/step - loss: 0.3839 - accuracy: 0.8286 - val_loss: 0.3781 - val_accuracy: 0.8272
Epoch 

In [22]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [23]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()